# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('city_data.csv')
df = pd.DataFrame(df)
df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temperature (F),Wind Speed
0,Témiscaming,CA,1586914752,46.72,-79.10,75,92,30.00,9.17
1,Mahébourg,MU,1586914694,-20.41,57.70,75,74,78.80,16.11
2,Tuktoyaktuk,CA,1586914656,69.45,-133.04,75,86,26.60,8.05
3,Dingcheng,CN,1586914753,32.13,115.04,0,51,72.93,11.12
4,Virú,PE,1586914753,-8.42,-78.75,24,70,74.53,5.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = df[["Latitude","Longitude"]]

# Store Humidity in weight
weight = df['Humidity (%)']

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

hotel_df = df.loc[(df['Max Temperature (F)']<80)&(df['Max Temperature (F)']>70)&(df['Cloudiness (%)']==0)&(df['Wind Speed']<10),:]
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temperature (F),Wind Speed
104,Bilma,NE,1586914771,18.69,12.92,0,15,77.77,6.17
231,Adrar,MR,1586914796,20.50,-10.07,0,50,74.66,6.82
481,Linqiong,CN,1586914568,30.42,103.46,0,77,71.01,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create empty string to store the info for first hotel later
hotel_df['Hotel'] = ""

# geocoordinates
#target_coordinates = hotel_df[['Latitude','Longitude']]
#target_coordinates = hotel_df.loc[:,['Latitude','Longitude']]
target_coordinates = ['18.69, 12.92','20.50, -10.07','30.42, 103.46']
#target_coordinates = '18.69, 12.92'
#target_coordinates = '20.50, -10.07'
#target_coordinates = '30.42, 103.46'
## List comprehension to generate the lat/longitudes?

target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    #"radius": target_radius,
    'rankby':'distance',
    "type": target_type,
    "key": g_key
}

#Method 1: using params dictionary
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#response = requests.get(base_url, params=params).json()

#Method 2: using full url
# url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&location={target_coordinates}&rankby=distance&type=hotel'
# response = requests.get(url).json()

response = []
for coordinate in target_coordinates:
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&location={coordinate}&rankby=distance&type=hotel'
    response.append(requests.get(url).json())


from pprint import pprint
pprint(response)

C:\Users\jakec\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[{'html_attributions': [],
  'results': [{'geometry': {'location': {'lat': 18.6880347, 'lng': 12.9194761},
                            'viewport': {'northeast': {'lat': 18.69126255,
                                                       'lng': 12.9204250802915},
                                         'southwest': {'lat': 18.68695875,
                                                       'lng': 12.9177271197085}}},
               'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/worship_islam-71.png',
               'id': '64ab20aa5d64dcdb426cf116aa95b4a0453cf870',
               'name': 'Централна джамия Билма',
               'place_id': 'ChIJHzuibjVccBERwZNx6j7YDNs',
               'plus_code': {'compound_code': 'MWQ9+6Q Bilma, Niger',
                             'global_code': '7FCJMWQ9+6Q'},
               'rating': 3.8,
               'reference': 'ChIJHzuibjVccBERwZNx6j7YDNs',
               'scope': 'GOOGLE',
               'types': ['mosque',
                      

In [7]:
#Pull out any relevant info i.e. lat/longitude, hotel name
lat1 = response[0]['results'][0]['geometry']['location']['lat']
lng1 = response[0]['results'][0]['geometry']['location']['lng']
name1 = response[0]['results'][0]['name']
lat3 = response[2]['results'][0]['geometry']['location']['lat']
lng3 = response[2]['results'][0]['geometry']['location']['lng']
name3 = response[2]['results'][0]['name']

nearest_hotel = pd.DataFrame({'Name':[name1,name3],
                              'Lat':[lat1,lat3],
                              'Lng':[lng1,lng3]})
nearest_hotel

##Note: there were no results for my second list of coordinates


,Name,Lat,Lng
0,Централна джамия Билма,18.688035,12.919476
1,Ludao Famous Garden Parking Lot,30.420503,103.459957


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
# Create a map using state centroid coordinates to set markers
marker_locations = nearest_hotel[['Lat', 'Lng']]

# Display Map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))